# LCEL 체인을 통한 챗봇 구성

## OpenAI LLM 준비 
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# .env 파일 로드
load_dotenv()

print(f"{os.getenv("OPENAI_API_KEY")[:9]}***")

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print(llm.model_name)

sk-proj-Y***
gpt-4o-mini



## 프롬프트 메시지 구성
역할 기반 프롬프트 템플릿 정의
- system 역할: LLM에게 '명탐정 코난 전문가' 역할 부여
- human 역할: 사용자 질문 삽입

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 '명탐정 코난' 전문가입니다. 모든 질문에 친절하고 정확하게 답해주세요."),
    ("human", "{question}")
])

## 체인 구성 및 UI 입출력 함수 정의

### LCEL 체인 구성 

체인 (프롬프트 → LLM → 출력 파서)를 파이프(|)로 연결
- `ChatPromptTemplate`: 문자열 메시지 생성
- `LLM`: 응답 생성
- `StrOutputParser`: LLM 응답에서 텍스트 추출

### Gradio 입출력 함수 정의
- `conan_expert_reply` : 사용자 입력값을 받아 LCEL 체인에 전달하고 결과 리턴


In [6]:
# LCEL 체인 구성
chain = prompt | llm | StrOutputParser()

# Gradio 입출력 함수 정의
def conan_expert_reply(user_input):
    return chain.invoke({"question": user_input})

## Gradio UI 구성
Blocks()로 구성된 Gradio 앱 생성
- `Markdown`: 상단 타이틀
- `Textbox`: 질문 입력
- `Textbox`: 답변 출력
- `submit()`: 질문 입력 후 응답 출력

In [7]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️‍♂️ 명탐정 코난 전문가에게 질문해보세요!")
    question = gr.Textbox(label="질문을 입력하세요")
    answer = gr.Textbox(label="코난 전문가의 답변", lines=5)
    question.submit(fn=conan_expert_reply, inputs=question, outputs=answer)

# Gradio 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


-----
** End of Documents **